# Team 5 💣
1. Maximus Lee
2. Aloysius Woo
3. Lim Huai Fu
4. Tan Ai Xin
5. Jin Zhenglong

## Setup

In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

### Install Environment
Est Time: 8 mins



In [ ]:
#@title Python

%cd /content
!sudo apt-get install python3.8
!sudo apt-get install python3.8-distutils

!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip
!apt-get install libcairo2-dev
!apt update

In [ ]:
#@title GitHub Repos

# Empty default folder
!find /content -mindepth 1 -delete

# nice try
!git clone https://github.com/maximus-lee-678/ict3104_team_05.git .

In [ ]:
#@title FollowYourPose/MMPose Folder Scaffolding

# FollowYourPose
%cd /content/FollowYourPose
%mkdir /content/FollowYourPose/checkpoints

# MMPose
%cd /content/MMPose
%mkdir video_input
%mkdir video_output

In [ ]:
#@title FollowYourPose/MMPose Libaries
# https://github.com/open-mmlab/mmpose
# https://blog.csdn.net/qq_21532607/article/details/130226728
# https://colab.research.google.com/github/open-mmlab/mmpose/blob/master/demo/MMPose_Tutorial.ipynb

%cd /content
!python3.8 -m pip install -r other_files/requirements/requirements_colab.txt

!python3.8 -m mim install mmengine
!python3.8 -m mim install mmpose
!python3.8 -m mim install "mmcv>=2.0.0"
!python3.8 -m mim install "mmdet>=3.0.0"

### Mount Drive 🗻

1. Run the first cell to authenicate
2. Click on the URL of the Failure in the output Failure("Error opening URL: ...")
3. Select the ICT3104 shared account
4. Once successful, you can close the tab
5. Run the next cell
6. Done

In [ ]:
# @title First Cell
%cd /content
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
# @title Next Cell
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/gdrive/MyDrive

### Symbolic Linking

In [ ]:
# Root Directory
ROOT_DIR_PATH = "/content"

# Video
VIDEO_DIR_PATH = f"{ROOT_DIR_PATH}/video"
%mkdir {VIDEO_DIR_PATH}
VIDEO_SKELETON_DIR_PATH = f"{ROOT_DIR_PATH}/video/Skeleton"
%mkdir {VIDEO_SKELETON_DIR_PATH}

# I/O Files
TRAINING_CONTENT_DIR_PATH = f"{ROOT_DIR_PATH}/training_content"
%mkdir {TRAINING_CONTENT_DIR_PATH}
CUSTOM_MODEL_DIR_PATH = f"{ROOT_DIR_PATH}/custom_model"
%mkdir {CUSTOM_MODEL_DIR_PATH}
INFERENCE_OUTPUT_DIR_PATH = f"{ROOT_DIR_PATH}/inference_output"
%mkdir {INFERENCE_OUTPUT_DIR_PATH}

MMPOSE_DIR_PATH = f"{ROOT_DIR_PATH}/MMPose"
FYP_DIR_PATH = f"{ROOT_DIR_PATH}/FollowYourPose"

# FYP Config
CONFIG_DIR_PATH = f"{FYP_DIR_PATH}/configs"
%mkdir {CONFIG_DIR_PATH}

In [ ]:
# symbolic link edition
!ln -s /content/gdrive/MyDrive/ICT3104/video/* /content/video

# copy edition
# %cp -r /content/gdrive/MyDrive/ICT3104/video/* /content/video/

In [ ]:
# Checkpoints folder

# symbolic link edition
!ln -s /content/gdrive/MyDrive/ICT3104/checkpoints/* /content/FollowYourPose/checkpoints

# copy edition
# %cp -r /content/gdrive/MyDrive/ICT3104/checkpoints/* /content/FollowYourPose/checkpoints/

In [ ]:
# Charade Lookup Files
%mkdir /content/charades_lookup/

# symbolic link edition
!ln -s /content/gdrive/MyDrive/ICT3104/charades_lookup/* /content/charades_lookup

### Pre-setup

In [ ]:
import sys

# Check if the path is not already in sys.path before appending
path_to_append = '/usr/local/lib/python3.8/dist-packages'
if path_to_append not in sys.path:
    sys.path.append(path_to_append)

import os
import csv
import datetime
from omegaconf import OmegaConf

import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, Video, clear_output

## Video Browsing

In [ ]:
# @title Video Selection { display-mode: "form" }

# Helper Functions
def getFolderContent(folder_name):
  subfolders = []
  for content in os.listdir(VIDEO_DIR_PATH):
        content_path = os.path.join(VIDEO_DIR_PATH, content)

        # Check if it's a directory and not hidden
        if os.path.isdir(content_path) and not content.startswith("."):
            subfolders.append(content)

  return subfolders

# Init
video_dir_folders = os.listdir(VIDEO_DIR_PATH)
video_dir_folder_content = getFolderContent(video_dir_folders)

# Create layout
layout_single_button = widgets.Layout(width='212px',margin='0px 0px 20px 90px')
layout_double_button = widgets.Layout(width='104px')
layout_hbox = widgets.Layout(margin='0px 0px 0px 88px')
layout_output = widgets.Layout(margin='0px 0px 20px 0px', display='flex', align_items='flex-start')

# Create widgets
video_output_placeholder = widgets.Output(layout=layout_output)
video_subdir_dropdown = widgets.Dropdown(options=video_dir_folder_content, description='Folder:', value=None, disabled=False)
video_dropdown = widgets.Dropdown(options=[], description='Video:', disabled=True)
video_display_button = widgets.Button(description="Display", disabled=True, layout=layout_double_button)
video_refresh_button = widgets.Button(description="Refresh", disabled=False, layout=layout_double_button)
video_mmpose_button = widgets.Button(description="MMPose", disabled=True, layout=layout_double_button)
video_fyp_button = widgets.Button(description="FYP", disabled=True, layout=layout_double_button)

video_hbox_1 = widgets.HBox([video_display_button, video_refresh_button], layout=layout_hbox)
video_hbox_2 = widgets.HBox([video_mmpose_button, video_fyp_button], layout=layout_hbox)

video_output_placeholder_content = HTML("""
  <div style="width: 512px; height: 512px; border-radius: 5%; background-color: black; margin: 0 auto; display: flex; justify-content: center; align-items: center;">
      <div style="width: 500px; height: 500px; border-radius: 5%; border: 2px solid white;" />
  </div>
""")

# Create listeners
## Update video dropdown options based on the selected folder
def video_subdir_select(change):
    selected_video_folder = video_subdir_dropdown.value

    if selected_video_folder != None:
      selected_VIDEO_DIR_PATH = f"{VIDEO_DIR_PATH}/{selected_video_folder}"
      selected_video_dir_content = [file for file in os.listdir(selected_VIDEO_DIR_PATH) if file.endswith('.mp4')]
    else:
      selected_video_dir_content = []

    video_dropdown.options = selected_video_dir_content
    if not selected_video_dir_content:
        video_dropdown.disabled = False
        video_dropdown.value = None
    else:
        video_dropdown.disabled = False

## Display the selected video
def display_selected_video(change):
    selected_video = video_dropdown.value
    selected_video_folder = video_subdir_dropdown.value

    if selected_video:
        video_path = f"{VIDEO_DIR_PATH}/{selected_video_folder}/{selected_video}"
        video_display = Video(video_path, width=512, height=512, embed=True)

        # Clear the output placeholder and display the video
        with video_output_placeholder:
            clear_output()
            display(video_display)

## Refresh folder and directory
def refresh_folder_and_directory(change):
    video_dir_folders = os.listdir(VIDEO_DIR_PATH)
    video_dir_folder_content = getFolderContent(video_dir_folders)

    video_dropdown.options = []
    video_dropdown.value = None

    video_subdir_dropdown.options = video_dir_folder_content
    video_subdir_dropdown.value = None



## Enable button when a valid video is picked
def enable_button(change):
    if video_dropdown.value:
        video_display_button.disabled = False
        video_mmpose_button.disabled = False
        video_fyp_button.disabled = False
    else:
        video_display_button.disabled = True
        video_mmpose_button.disabled = True
        video_fyp_button.disabled = True

# Attach Listeners
video_subdir_dropdown.observe(video_subdir_select, 'value')
video_display_button.on_click(display_selected_video)
video_refresh_button.on_click(refresh_folder_and_directory)
video_dropdown.observe(enable_button, 'value')


# Display fields
with video_output_placeholder:
  display(video_output_placeholder_content)
display(video_output_placeholder)
display(video_subdir_dropdown)
display(video_dropdown)

display(video_hbox_1)
display(video_hbox_2)


## Inference: MMPOSE

In [ ]:
# @title Settings { display-mode: "form" }

# Helper Functions
## Re-encode video due to H.264 video encoding error
def reencode_video(input_file):
  temp_output_file = f"{VIDEO_SKELETON_DIR_PATH}/output.mp4"
  !ffmpeg -i {input_file} -c:v libx264 -crf 23 -c:a aac -strict experimental {temp_output_file}

  os.remove(input_file)
  os.rename(temp_output_file, input_file)

# Create layout
layout_single_long_button = widgets.Layout(margin='0px 0px 20px 154px')

# Create widgets
selected_mmpose_video_input = widgets.Text(placeholder='Select a video above', description="Video:", disabled=True)
inf_mmpose_button = widgets.Button(description="Start Inference", disabled=True, layout=layout_single_long_button)

# Create listeners
## Retrieve video input
def update_mmpose_input_video(change):
    selected_video = video_dropdown.value
    selected_mmpose_video_input.value = selected_video
    if selected_video:
        inf_mmpose_button.disabled = False
    else:
        inf_mmpose_button.disabled = True

## Run mmpose
def run_mmpose_inference(button):
    selected_video = video_dropdown.value
    video_path = f"/content/video/{video_subdir_dropdown.value}/{selected_video}"

    %cd -q {MMPOSE_DIR_PATH}

    if not os.path.exists(VIDEO_SKELETON_DIR_PATH):
        os.mkdir(VIDEO_SKELETON_DIR_PATH)

    # Start inference
    !python demo/inferencer_demo.py \
        {video_path}  \
        --pose2d human --vis-out-dir {VIDEO_SKELETON_DIR_PATH} \
        --black-background

    input_file = f"{VIDEO_SKELETON_DIR_PATH}/{selected_video}"
    reencode_video(input_file)

    clear_output()
    mmpose_inf_display()
    print(f"Done [Output: {input_file}]")

# Attach Listeners
video_mmpose_button.on_click(update_mmpose_input_video)
inf_mmpose_button.on_click(run_mmpose_inference)

# Display fields
def mmpose_inf_display():
  display(selected_mmpose_video_input)
  display(inf_mmpose_button)
mmpose_inf_display()

## Inference: FYP

In [ ]:
from ipywidgets import Dropdown, Output, Layout, widgets, Button, VBox, HBox
import os
from IPython.display import display, clear_output
from omegaconf import OmegaConf
import yaml


# Define the options for boolean dropdown
options = [True, False]


# Get a list of all files in the directory
config_files = [f for f in os.listdir(CONFIG_DIR_PATH) if os.path.isfile(os.path.join(CONFIG_DIR_PATH, f))]


# Create a dropdown widget with the list of config files
config_dropdown = Dropdown(
    options=["- Select an Item -"] + config_files,
    description='Select a Config File:',
    layout=Layout(width="500px"),
    style={'description_width': '150px'}
)

# Init
%cd -q {FYP_DIR_PATH}
sample_yaml_path = f"{CONFIG_DIR_PATH}/pose_sample.yaml"
load_config = OmegaConf.load(sample_yaml_path)


# Initialize load_config with a default value
load_config = None

# Define a container for displayed widgets
displayed_widgets = []


# Function to clear displayed widgets (excluding the dropdown)
def clear_displayed_widgets():
    for widget in displayed_widgets:
        widget.close()
    displayed_widgets.clear()
    display(config_dropdown)  # Display the dropdown again

# Function to update the load_config variable based on the selected filename
def update_load_config(change):
    global load_config
    selected_filename = change.new
    if selected_filename and selected_filename != "- Select an Item -":
        clear_output(wait=True)  # Clear the output area
        clear_displayed_widgets()  # Clear previously displayed widgets


        sample_yaml_path = f"{CONFIG_DIR_PATH}/{selected_filename}"
        load_config = OmegaConf.load(sample_yaml_path)

        print(f"Editing config from: {sample_yaml_path}")


        # Create widgets for Default Values (same as before)
        ## default data
        config_subheader1 = widgets.HTML(value="<h3>Basic Data</h3>")

        # ====================================

        # Define the directory path where the models are stored
        # Create a list of model names and paths in the directory
        model_options = [("Default", f"{FYP_DIR_PATH}/checkpoints/stable-diffusion-v1-4")]

        for model_name in os.listdir(CUSTOM_MODEL_DIR_PATH):
            model_path = os.path.join(CUSTOM_MODEL_DIR_PATH, model_name)
            if os.path.isdir(model_path):
                model_options.append((model_name, f"{CUSTOM_MODEL_DIR_PATH}/{model_name}"))

        # Create the pretrained_model_path dropdown
        config_pretrained_model_path = Dropdown(
            options=model_options,
            description="pretrained_model_path:",
            value=load_config.pretrained_model_path,
            layout=config_layout,
            style=config_style
        )

        # Create an Output widget to display the selected path
        selected_path_output = Output()

        # Function to handle the change event
        def handle_change(change):
            selected_option = change.new
            if selected_option:
                selected_path = selected_option[0] if selected_option[1] != "Default" else selected_option[0]
                with selected_path_output:
                    selected_path_output.clear_output()


        # Attach the event handler to the dropdown's 'value' trait
        config_pretrained_model_path.observe(handle_change, names='value')
        display(selected_path_output)

        # =====================================

        config_output_dir_name = widgets.Text(description="output_dir_name:", value="", style=config_style, layout=config_layout)

        # =====================================
        config_train_batch_size = widgets.IntText(description="train_batch_size:", value=load_config.train_batch_size, style=config_style, layout=config_layout)
        config_validation_steps = widgets.IntText(description="validation_steps:", value=load_config.validation_steps, style=config_style, layout=config_layout)
        config_seed = widgets.IntText(description="seed:", value=load_config.seed, style=config_style, layout=config_layout)
        config_mixed_precision = widgets.Text(description="mixed_precision:", value=load_config.mixed_precision, style=config_style, layout=config_layout)
        config_gradient_checkpointing = Dropdown(options=options, value=True if load_config.gradient_checkpointing else False, description="gradient_checkpointing:", layout=config_layout, style=config_style)
        config_enable_xformers_memory_efficient_attention = Dropdown(options=options, value=True if load_config.enable_xformers_memory_efficient_attention else False, description="enable_xformers_memory_efficient_attention:", layout=config_layout, style=config_style)
        ## validation_data
        config_subheader2 = widgets.HTML(value="<h3>Validation Data</h3>")
        config_prompts = widgets.Textarea(description="prompts:", value="\n".join(load_config.validation_data.prompts), style=config_style, layout=widgets.Layout(width="500px", height="100px"))
        config_video_length = widgets.IntText(description="video_length:", value=load_config.validation_data.video_length, style=config_style, layout=config_layout)
        config_width = widgets.IntText(description="width:", value=load_config.validation_data.width, style=config_style, layout=config_layout)
        config_height = widgets.IntText(description="height:", value=load_config.validation_data.height, style=config_style, layout=config_layout)
        config_num_inference_steps = widgets.IntText(description="num_inference_steps:", value=load_config.validation_data.num_inference_steps, style=config_style, layout=config_layout)
        config_guidance_scale = widgets.FloatText(description="guidance_scale:", value=load_config.validation_data.guidance_scale, style=config_style, layout=config_layout)
        config_use_inv_latent = Dropdown(options=options, value=True if load_config.validation_data.use_inv_latent else False, description="use_inv_latent:", layout=config_layout, style=config_style)
        config_num_inv_steps = widgets.IntText(description="num_inv_steps:", value=load_config.validation_data.num_inv_steps, style=config_style, layout=config_layout)
        config_num_dataset_set = widgets.Text(description="dataset_set:", value=load_config.validation_data.dataset_set, style=config_style, layout=config_layout)
        ## Button Widget
        config_save_btn = widgets.Button(description="Save", layout=config_button_layout)
        ## Group widgets
        config_vbox = widgets.VBox([
            config_pretrained_model_path,
            config_output_dir_name,
            config_train_batch_size,
            config_validation_steps,
            config_seed,
            config_mixed_precision,
            config_gradient_checkpointing,
            config_enable_xformers_memory_efficient_attention,
        ])
        config_vbox_validation_data = widgets.VBox([
            config_prompts,
            config_video_length,
            config_width,
            config_height,
            config_num_inference_steps,
            config_guidance_scale,
            config_use_inv_latent,
            config_num_inv_steps,
            config_num_dataset_set
        ])

        # Display fields (same as before)
        display(config_subheader1)
        display(config_vbox)
        display(config_subheader2)
        display(config_vbox_validation_data)
        display(config_save_btn)

        # Create listeners
        def save_config(change):
          outputdir_path = "/content/inference_output" + "/"
          config = {
              "pretrained_model_path": config_pretrained_model_path.value,
              "output_dir": outputdir_path + config_output_dir_name.value,
              "validation_data": {
                  "prompts": [prompt.strip() for prompt in config_prompts.value.splitlines() if prompt.strip()],
                  "video_length": config_video_length.value,
                  "width": config_width.value,
                  "height": config_height.value,
                  "num_inference_steps": config_num_inference_steps.value,
                  "guidance_scale": config_guidance_scale.value,
                  "use_inv_latent": config_use_inv_latent.value,
                  "num_inv_steps": config_num_inv_steps.value,
                  "dataset_set": config_num_dataset_set.value
              },
              "train_batch_size": config_train_batch_size.value,
              "validation_steps": config_validation_steps.value,
              "resume_from_checkpoint": load_config.resume_from_checkpoint,
              "seed": config_seed.value,
              "mixed_precision": config_mixed_precision.value,
              "gradient_checkpointing": config_gradient_checkpointing.value,
              "enable_xformers_memory_efficient_attention": config_enable_xformers_memory_efficient_attention.value
          }
          # OmegaConf.save(config, sample_yaml_path)
          with open(sample_yaml_path, "w") as file:
            yaml.dump(config, file, default_style='"', default_flow_style=False, sort_keys=False)
          print("Successfully saved!")

        # Attach Listeners
        config_save_btn.on_click(save_config)

        # Update the displayed_widgets list
        displayed_widgets.extend([
            config_subheader1,
            config_vbox,
            config_subheader2,
            config_vbox_validation_data,
            config_save_btn,
        ])


# Attach the event handler to the dropdown's 'value' trait
config_dropdown.observe(update_load_config, names='value')


# Create layout
config_style = {'description_width': '150px'}
config_layout = widgets.Layout(width="500px")
config_button_layout = widgets.Layout(margin='0px 0px 20px 354px')


# Create an Output widget to display messages and results
output_widget = Output()


# Display the dropdown widget and the output widget
display(config_dropdown)
display(output_widget)


In [ ]:
# @title Settings { display-mode: "form" }

# Create layout
layout_single_long_button = widgets.Layout(margin='0px 0px 20px 154px')

# Create widgets
selected_fyp_video_input = widgets.Text(placeholder='Select a video above', description="Video:", disabled=True)
config_file_path = widgets.Text(value='configs/pose_sample.yaml',description="Config File:")
inf_fyp_button = widgets.Button(description="Start Inference", disabled=True, layout=layout_single_long_button)

# Create listeners
## Retrieve video input
def update_fyp_input_video(change):
    selected_video = video_dropdown.value
    selected_fyp_video_input.value = f"../video/{video_subdir_dropdown.value}/{selected_video}"
    if selected_video:
        inf_fyp_button.disabled = False
    else:
        inf_fyp_button.disabled = True

## Run FYP
def run_fyp_inference(button):
    config_file_path_text = config_file_path.value
    video_file_path_text = selected_fyp_video_input.value

    %cd {FYP_DIR_PATH}

    # Start inference
    !TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py \
        --config={config_file_path_text}  \
        --skeleton_path={video_file_path_text}

# Attach Listeners
inf_fyp_button.on_click(run_fyp_inference)
video_fyp_button.on_click(update_fyp_input_video)

# Display fields
display(selected_fyp_video_input)
display(config_file_path)
display(inf_fyp_button)

## Training (WIP)

In [ ]:
# @title Init { display-mode: "form" }

# Charades Data Class from csv
class CharadesData:
  def __init__(self, row):
    id, subject, scene, quality, relevance, verified, script, objects, descriptions, actions, length = row
    self.id = id
    self.subject = subject
    self.scene = scene
    self.quality = quality
    self.relevance = relevance
    self.verified = verified
    self.script = script
    self.objects = objects.split(";")
    self.descriptions = descriptions
    self.length = length
    self.actions = {}

    # Convert actions in proper data structure ("class_id time_start time_end" -> class_id: [time_start, time_end])
    if len(actions) != 0:
      action_substrings = actions.split(';')
      for substring in action_substrings:
        parts = substring.split()
        key = parts[0]
        values = [self.convert_to_ms(parts[1]), self.convert_to_ms(parts[2])]
        self.actions[key] = values

  # For printing
  def __str__(self):
        return f"ID: {self.id}, Subject: {self.subject}, Scene: {self.scene}, Quality: {self.quality}, Relevance: {self.relevance}, Verified: {self.verified}, Script: {self.script}, Objects: {self.objects}, Descriptions: {self.descriptions}, Actions: {self.actions}, Length: {self.length}"

  # Helper function to convert time into ms
  def convert_to_ms(self, seconds):
    ss,ms = seconds.split('.')
    total_ms = 1000*int(ss) + int(ms)
    return total_ms

  # Caption getter with template
  def getCaption(self, index):
    return f"In a {self.scene} setting, within the context of '{self.script}', the action '{action_descriptions[list(self.actions.keys())[index]]}' is taking place."


action_descriptions = {}
charades_all = []

# Load classes lookup table
with open('/content/charades_lookup/Charades_v1_classes.txt', 'r') as file:
    for line in file:
        code, description = line.strip().split(' ', 1)
        action_descriptions[code] = description

# Load charades data A
with open('/content/charades_lookup/Charades_v1_train.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader, None)

    for row in csv_reader:
        charadeData = CharadesData(row)
        charades_all.append(charadeData)

# Load charades data B
with open('/content/charades_lookup/Charades_v1_test.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader, None)

    for row in csv_reader:
        charadeData = CharadesData(row)
        charades_all.append(charadeData)

clear_output()
print("Data load successful!")

In [ ]:
# @title Dataset Preload (Video) { display-mode: "form" }

# Init
dataset_dir_folders = os.listdir(VIDEO_DIR_PATH)

## Training env
training_dataset = None
training_idx = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
training_branch = f"{TRAINING_CONTENT_DIR_PATH}/{training_idx}"

# Create layout
config_style = {'description_width': '100px'}
config_layout = widgets.Layout(width="300px")
config_button_layout = widgets.Layout(margin='0px 0px 20px 154px')

# Create widgets
dataset_folder_dropdown = widgets.Dropdown(options=[dir for dir in dataset_dir_folders if not dir.startswith(".")], description='Dataset Folder:', value=None, layout=config_layout, style=config_style)
dataset_cutting_button = widgets.Button(description="Start Cutting", disabled=True, layout=config_button_layout)

# Create listeners
def dataset_dir_select(change):
  if dataset_folder_dropdown:
    dataset_folder_content = os.listdir(f"{VIDEO_DIR_PATH}/{dataset_folder_dropdown.value}")
    total_dataset_videos = len([file for file in dataset_folder_content if file.endswith('.mp4')])

    dataset_cutting_button.disabled = False
    dataset_cutting_button.description = f"Start Cutting ({total_dataset_videos})"

def video_cutting_select(change):
  os.makedirs(training_branch, exist_ok=True)
  training_dataset = f"{VIDEO_DIR_PATH}/{dataset_folder_dropdown.value}"

  # Loop video files from selected dataset folder
  for video_file in os.listdir(training_dataset):
    video, ext = os.path.splitext(video_file)

    # Ignore non video files (Eg: .ipynb_checkpoint and csv)
    if ext != ".mp4":
      continue

    video_folder = f"{training_branch}/{video}"
    if not os.path.exists(video_folder):
      os.mkdir(video_folder)

      # Retrieve charade object by ID
      charade_data = None
      for charade in charades_all:
        if charade.id == video:
            charade_data = charade
            break

      # If no clipping required, keep whole video
      if not charade_data.actions:
        print(f"No clipping needed")
      else:
        print(f"Clipping {video}")
        charade_actions = charade_data.actions.items()
        total_charade_actions = len(charade_actions)
        for i, (class_id, timings) in enumerate(charade_actions):

          input_video = f"{training_dataset}/{video_file}"
          output_video = f"{video_folder}/{video}{i+1:02}{ext}"

          print(f"#{i+1}/{total_charade_actions}: {timings[0]}ms to {timings[0]+timings[1]}ms [I:{input_video}] [O:{output_video}]")
          !ffmpeg -i {input_video} -ss {timings[0]}ms -t {timings[1]}ms -c:v libx264 -c:a aac {output_video} -loglevel quiet
    else:
      print(f"Folder already exist for video_id: {video}. Skipping ...")
  print("Finished Clipping")

# Attach Listeners
dataset_folder_dropdown.observe(dataset_dir_select, 'value')
dataset_cutting_button.on_click(video_cutting_select)

# Display fields
display(dataset_folder_dropdown)
display(dataset_cutting_button)

In [ ]:
# @title Dataset Preload (Metadata) { display-mode: "form" }

training_metadata_file = f"{training_branch}/metadata.tsv"
with open(training_metadata_file, 'w', newline='', encoding='utf-8') as tsvfile:
  fieldnames = ['part_id', 'clip_id', 'caption']
  writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
  writer.writeheader()

  for part_id in os.listdir(training_branch):
    folder_path = os.path.join(training_branch, part_id)

    # Ignore non video files (Eg: .ipynb_checkpoint and csv)
    if not os.path.isdir(folder_path) or part_id.startswith("."):
      continue

    charade_data = None
    for charade in charades_all:
      if charade.id == part_id:
          charade_data = charade
          break

    if not charade_data:
      print("Missing charades data, skipping ...")
      continue

    # Sort by video sub-id to maintain order
    training_video_files = sorted(os.listdir(folder_path), key=lambda x: int(os.path.splitext(x)[0][-2:]))
    for i, clip in enumerate(training_video_files):
      caption = charade_data.getCaption(i)
      writer.writerow({
          'part_id': part_id,
          'clip_id': clip,
          'caption': caption
      })
print(f"TSV created: {training_metadata_file}")

In [ ]:
# @title Training File Configuration { display-mode: "form" }

from ipywidgets import Dropdown, Output, Layout, widgets, Button, VBox, HBox
import os
from IPython.display import display, clear_output
from omegaconf import OmegaConf
import yaml


# Init
%cd -q {FYP_DIR_PATH}

# Initialize load_config with a default value
load_config = None

# Define the options for boolean dropdown
options = [True, False]


# Get a list of all files in the directory
config_files = [f for f in os.listdir(CONFIG_DIR_PATH) if os.path.isfile(os.path.join(CONFIG_DIR_PATH, f))]


# Create a dropdown widget with the list of config files
config_dropdown = Dropdown(
    options=["- Select an Item -"] + config_files,
    description='Select a Config File:',
    layout=Layout(width="500px"),
    style={'description_width': '150px'}
)

# Define a container for displayed widgets
displayed_widgets = []


# Function to clear displayed widgets (excluding the dropdown)
def clear_displayed_widgets():
    for widget in displayed_widgets:
        widget.close()
    displayed_widgets.clear()
    display(config_dropdown)  # Display the dropdown again

# Function to update the load_config variable based on the selected filename
def update_load_config(change):
    global load_config
    selected_filename = change.new
    if selected_filename and selected_filename != "- Select an Item -":
        clear_output(wait=True)  # Clear the output area
        clear_displayed_widgets()  # Clear previously displayed widgets


        sample_yaml_path = f"{CONFIG_DIR_PATH}/{selected_filename}"
        load_config = OmegaConf.load(sample_yaml_path)

        print(f"Editing config from: {sample_yaml_path}")


        # Create widgets for Default Values (same as before)
        ## default data
        config_subheader1 = widgets.HTML(value="<h3>Basic Data</h3>")

        # ====================================

        # Create a list of model names and paths in the directory
        model_options = [("Default", f"{FYP_DIR_PATH}/checkpoints/stable-diffusion-v1-4")]

        for model_name in os.listdir(CUSTOM_MODEL_DIR_PATH):
            model_path = os.path.join(CUSTOM_MODEL_DIR_PATH, model_name)
            if os.path.isdir(model_path):
                model_options.append((model_name, f"{CUSTOM_MODEL_DIR_PATH}/{model_name}"))

        # Create the pretrained_model_path dropdown
        config_pretrained_model_path = Dropdown(
            options=model_options,
            description="pretrained_model_path:",
            value=load_config.pretrained_model_path,
            layout=config_layout,
            style=config_style
        )

        # =====================================
        
        config_output_dir_name = widgets.Text(description="output_dir_name:", value="", style=config_style, layout=config_layout)

        # =====================================
        config_learning_rate = widgets.FloatText(description="learning_rate:", value=load_config.learning_rate, style=config_style, layout=config_layout)
        config_train_batch_size = widgets.IntText(description="train_batch_size:", value=load_config.train_batch_size, style=config_style, layout=config_layout)
        config_max_train_steps = widgets.IntText(description="max_train_steps:", value=load_config.max_train_steps, style=config_style, layout=config_layout)
        # =====================================
        config_seed = widgets.IntText(description="seed:", value=load_config.seed, style=config_style, layout=config_layout)
        config_mixed_precision = widgets.Text(description="mixed_precision:", value=load_config.mixed_precision, style=config_style, layout=config_layout)
        config_use_8bit_adam = Dropdown(options=options, value=load_config.use_8bit_adam, description="config_use_8bit_adam:", layout=config_layout, style=config_style)
        config_gradient_checkpointing = Dropdown(options=options, value=load_config.gradient_checkpointing, description="gradient_checkpointing:", layout=config_layout, style=config_style)
        config_enable_xformers_memory_efficient_attention = Dropdown(options=options, value=load_config.enable_xformers_memory_efficient_attention, description="enable_xformers_memory_efficient_attention:", layout=config_layout, style=config_style)
       # =====================================
        # train_data
        config_subheader2 = widgets.HTML(value="<h3>Train Data</h3>")
        config_video_path = widgets.Text(description="video_path:", value=load_config.train_data.video_path, style=config_style, layout=config_layout)
        config_n_sample_frames = widgets.IntText(description="n_sample_frames:", value=load_config.train_data.n_sample_frames, style=config_style, layout=config_layout)
        config_train_data_width = widgets.IntText(description="width:", value=load_config.train_data.width, style=config_style, layout=config_layout)
        config_sample_frame_rate = widgets.IntText(description="sample_frame_rate:", value=load_config.train_data.sample_frame_rate, style=config_style, layout=config_layout)
        # =====================================
        ## Button Widget
        config_save_btn = widgets.Button(description="Save", layout=config_button_layout)
        ## Group widgets
        config_vbox = widgets.VBox([
            config_pretrained_model_path,
            config_output_dir_name,
            config_learning_rate,
            config_train_batch_size,
            config_max_train_steps,
            config_seed,
            config_mixed_precision,
            config_use_8bit_adam,
            config_gradient_checkpointing,
            config_enable_xformers_memory_efficient_attention
        ])
        config_vbox_train_data = widgets.VBox([
            config_video_path,
            config_n_sample_frames,
            config_train_data_width,
            config_sample_frame_rate,
        ])

        # Display fields (same as before)
        display(config_subheader1)
        display(config_vbox)
        display(config_subheader2)
        display(config_vbox_train_data)
        display(config_save_btn)

        # Create listeners
        def save_config(change):

          config = {
              "pretrained_model_path": config_pretrained_model_path.value,
              "output_dir": f"{CUSTOM_MODEL_DIR_PATH}/{config_output_dir_name.value}",

              "train_data": {
                  "video_path": config_video_path.value,
                  "n_sample_frames": config_n_sample_frames.value,
                  "width": config_train_data_width.value,
                  "sample_frame_rate": config_sample_frame_rate.value,
              },

              "learning_rate": config_learning_rate.value,
              "train_batch_size": config_train_batch_size.value,
              "max_train_steps": config_max_train_steps.value,
              "trainable_modules": list(load_config.trainable_modules),

              "seed": config_seed.value,
              "mixed_precision": config_mixed_precision.value,
              "use_8bit_adam": config_use_8bit_adam.value,
              "gradient_checkpointing": config_gradient_checkpointing.value,
              "enable_xformers_memory_efficient_attention": config_enable_xformers_memory_efficient_attention.value
          }
          with open(sample_yaml_path, "w") as file:
            yaml.dump(config, file, default_style='"', default_flow_style=False, sort_keys=False)
          print("Successfully saved!")

        # Attach Listeners
        config_save_btn.on_click(save_config)

        # Update the displayed_widgets list
        displayed_widgets.extend([
            config_subheader1,
            config_vbox,
            config_subheader2,
            config_vbox_train_data,
            config_save_btn,
        ])


# Attach the event handler to the dropdown's 'value' trait
config_dropdown.observe(update_load_config, names='value')

# Create layout
config_style = {'description_width': '150px'}
config_layout = widgets.Layout(width="500px")
config_button_layout = widgets.Layout(margin='0px 0px 20px 354px')


# Create an Output widget to display messages and results
output_widget = Output()


# Display the dropdown widget and the output widget
display(config_dropdown)
display(output_widget)


In [ ]:
# Changed fields in pose_train
## output_dir: "/content/custom_model/<model_name>" [test01]
## video_path: "/content/training_content/<dataset_folder>" [20231007_161145]

%cd {FYP_DIR_PATH}
!TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch train_followyourpose.py --config="configs/pose_train.yaml"

In [ ]:
# @title Selection { display-mode: "form" }

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the base directory path
base_directory = '/content'

# Function to create nested dropdowns for directories
def create_directory_dropdown(directory_path, default_value='Select'):
    folder_picker = widgets.Dropdown(
        options=['Select'] + get_subdirectories(directory_path),
        value=default_value,
        description=os.path.basename(directory_path) + ':',
        disabled=False,
    )
    folder_picker.style.description_width = 'initial'  # Allow the description to shrink
    folder_picker.layout.width = 'auto'  # Allow the dropdown to adjust its width
    folder_picker.layout.min_width = '200px'  # Set a minimum width to ensure content is visible
    folder_picker.layout.width = '400px'
    return folder_picker

# Function to create "Training Epochs" textbox
def create_epochs_textbox(default_value=10):
    epochs_textbox = widgets.IntText(
        value=default_value,
        description='Training Epochs:',
        layout=widgets.Layout(width='auto')
    )
    epochs_textbox.style.description_width = 'initial'  # Allow the description to shrink
    epochs_textbox.layout.width = 'auto'  # Allow the textbox to adjust its width
    epochs_textbox.layout.min_width = '200px'  # Set a minimum width to ensure content is visible
    epochs_textbox.layout.width = '400px'
    return epochs_textbox

# Function to create "Batch Size" textbox
def create_batch_size_textbox(default_value=32):
    batch_size_textbox = widgets.IntText(
        value=default_value,
        description='Batch Size:',
        layout=widgets.Layout(width='auto')
    )
    batch_size_textbox.style.description_width = 'initial'  # Allow the description to shrink
    batch_size_textbox.layout.width = 'auto'  # Allow the textbox to adjust its width
    batch_size_textbox.layout.min_width = '200px'  # Set a minimum width to ensure content is visible
    batch_size_textbox.layout.width = '400px'
    return batch_size_textbox

# Function to create "Submit" button
def create_submit_button():
    submit_button = widgets.Button(
        description='Submit',
    )
    submit_button.layout.margin = '20px 0 0 200px'  # Add margin for positioning
    return submit_button

# Function to get subdirectories in a folder
def get_subdirectories(folder_path):
    folders = os.listdir(folder_path)
    return [content for content in folders if os.path.isdir(os.path.join(folder_path, content))]

# Function to display the selected values with full paths
def display_selected_values(selected_folder, subfolder, epochs, batch_size):
    clear_output(wait=True)
    dataset_folder_path = os.path.join(base_directory, selected_folder)
    subfolder_path = os.path.join(dataset_folder_path, subfolder) if subfolder else None
    print(f'Selected Dataset Folder: {dataset_folder_path}')
    if subfolder_path:
        print(f'Selected Dataset Subfolder: {subfolder_path}')
    print(f'Epochs Entered: {epochs}')
    print(f'Batch Size Entered: {batch_size}')

# Create an output widget as a placeholder
output_placeholder = widgets.Output()

# Create the "Dataset Folder" dropdown
dataset_folder_picker = create_directory_dropdown(base_directory)
dataset_folder_picker.description = 'Dataset Folder:'

# Create empty variables for the subfolder, epochs, and batch size widgets
subfolder_picker = None
epochs_textbox = None
batch_size_textbox = None
submit_button = None

# Function to handle button click
def on_submit_button_click(b):
    selected_folder = dataset_folder_picker.value
    subfolder = subfolder_picker.value if subfolder_picker else None
    epochs = epochs_textbox.value
    batch_size = batch_size_textbox.value
    display_selected_values(selected_folder, subfolder, epochs, batch_size)

# Function to update the directory dropdowns based on the selected "Dataset Folder"
def update_directory_dropdowns(change):
    global subfolder_picker, epochs_textbox, batch_size_textbox, submit_button

    selected_folder = change['new']

    # Clear previous subfolder, epochs, and batch size widgets
    if subfolder_picker:
        subfolder_picker.close()
    if epochs_textbox:
        epochs_textbox.close()
    if batch_size_textbox:
        batch_size_textbox.close()
    if submit_button:
        submit_button.close()

    # Create the "Dataset Subfolder" dropdown if a valid folder is selected
    if selected_folder != 'Select':
        subfolder_picker = create_directory_dropdown(os.path.join(base_directory, selected_folder), default_value='Select')
        subfolder_picker.description = 'Dataset Subfolder:'
        display(subfolder_picker)

        # Create "Training Epochs" and "Batch Size" textboxes
        epochs_textbox = create_epochs_textbox()
        batch_size_textbox = create_batch_size_textbox()
        submit_button = create_submit_button()
        submit_button.on_click(on_submit_button_click)

        # Display the "Submit" button
        display(epochs_textbox, batch_size_textbox, submit_button)

# Observe changes in the "Dataset Folder" dropdown
dataset_folder_picker.observe(update_directory_dropdowns, names='value')

# Display the widgets
display(dataset_folder_picker, output_placeholder)


In [ ]:
!ffmpeg -i /content/test/YWS08.mp4 -ss 00:00:02 -t 00:00:02 -c:v libx264 -c:a aac /content/test/output.mp4